In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(database="postgres",
                        host="localhost",
                        user="postgres",
                        password="dspproject123",
                        port="5432")

In [12]:
cursor = conn.cursor()
query = "select * from customer C, orders O, nation N where C.c_custkey = O.o_custkey and C.c_name like '%cheng' and C.c_nationkey = N.n_nationkey"
extract_qp = "EXPLAIN (COSTS FALSE, TIMING FALSE) "
cursor.execute(extract_qp + query)
extract_qp+query

"EXPLAIN (COSTS FALSE, TIMING FALSE) select * from customer C, orders O, nation N where C.c_custkey = O.o_custkey and C.c_name like '%cheng' and C.c_nationkey = N.n_nationkey"

In [13]:
qep_list1 = cursor.fetchall()

In [14]:
for i in qep_list1:
    print(i[0])

Gather
  Workers Planned: 2
  ->  Hash Join
        Hash Cond: (c.c_nationkey = n.n_nationkey)
        ->  Parallel Hash Join
              Hash Cond: (o.o_custkey = c.c_custkey)
              ->  Parallel Seq Scan on orders o
              ->  Parallel Hash
                    ->  Parallel Seq Scan on customer c
                          Filter: ((c_name)::text ~~ '%cheng'::text)
        ->  Hash
              ->  Seq Scan on nation n


In [7]:
from collections import deque


In [22]:
query = "select AVG(o_totalprice) from orders"
extract_qp = "EXPLAIN (ANALYZE false, SETTINGS true, FORMAT JSON) "
cursor.execute(extract_qp + query)

In [23]:
qep = cursor.fetchall()[0][0][0].get("Plan")

In [26]:
node_list = []
graph_str = ''''''
graph_str = graph_str + 'digraph {'
q = deque([qep])
while q:
    for i in range(len(q)):
        node = q.popleft()
        parent = node['Node Type']
        node_list.append(node)
        if "Plans" in node:
            for child in node['Plans']:
                graph_str = graph_str + parent + " -> " + child['Node Type'] + "\n"
                q.append(child)
graph_str = graph_str + '}'

In [27]:
graph_str

'digraph {Hash Join -> Seq Scan\nHash Join -> Hash\nHash -> Seq Scan\n}'

In [21]:
import graphviz
import streamlit as st
graph = '''
    digraph {{
        {e1} [color ="red"]
        {e1} -> {e2}
    }}
'''.format(e1 = 'element1', e2 = 'element2')

st.graphviz_chart(graph)

2023-03-29 20:20:00.115 
  command:

    streamlit run c:\Users\mdzak\anaconda3\envs\dsp2\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [17]:
str1 = "hello word"
str2 = "bye world"
new_string = str1 + "\n" + str2

In [18]:
new_string

'hello word\nbye world'

In [ ]:
node_list

In [ ]:
node_types

In [ ]:
node_type_list = []
count = 0
for line in qep:
    n_type = ''
    node = line[0].split()
    if node[0] == '->':
        try:
            n_type = node[1] + " " + node[2]
        except:
            n_type = node[1]
        node_type_list.append(n_type)
    elif count == 0:
        try:
            n_type = node[0] + " " + node[1]
        except:
            n_type = node[0]
        node_type_list.append(n_type)
        count = count + 1

In [ ]:
node_type_list

In [ ]:
exp_text = ""

In [ ]:
import psycopg2

# Connect to the PostgreSQL database
conn = psycopg2.connect(database="your_database_name", user="your_username", password="your_password", host="your_host", port="your_port")
cur = conn.cursor()

# Execute the query and retrieve the query plan
cur.execute("EXPLAIN ANALYZE your_query_here")
plan = cur.fetchall()

# Define a function to convert the plan to natural language
def explain_plan(plan):
    explanation = ""

    for line in plan:
        # Split the line into its components
        line_parts = line[0].split(" ")
        
        # Check if the line represents a node in the execution plan
        if line_parts[0] == "->":
            # Extract the node type and the estimated cost
            node_type = line_parts[1]
            estimated_cost = line_parts[-1].replace("..", "to")

            # Generate a natural language explanation based on the node type
            if node_type == "Seq Scan":
                explanation += "The database is scanning the entire table to retrieve the required data. "
            elif node_type == "Index Scan":
                explanation += "The database is using an index to retrieve the required data. "
            elif node_type == "Nested Loop":
                explanation += "The database is performing a nested loop join between two tables. "
            elif node_type == "Hash Join":
                explanation += "The database is performing a hash join between

In [24]:
def remove_lines_without_arrow(string):
    # split the string into a list of lines
    lines = string.split('\n')
    
    # create a new list of lines that contain the "->" symbol
    filtered_lines = [line for line in lines if "[" in line]
    
    # join the filtered lines back into a string
    new_string = '\n'.join(filtered_lines)
    lines = new_string.split('\n')
    
    # create a new list of text between the first two " characters in each line
    extracted_text = []
    for line in lines:
        start = line.find('"') + 1
        end = line.find('"', start)
        extracted_text.append(line[start:end])
    
    # join the extracted text back into a string
    new_string = '\n'.join(extracted_text)
    
    return new_string

In [58]:
g1 = '''digraph {
1[label="HashJoin"]
2[label="SeqScan"]
1->2
3[label="Hash"]
1->3
4[label="SeqScan"]
3->4
}
}'''

In [78]:
g2 = '''digraph {
1[label="Gather"]
2[label="HashJoin"]
1->2
3[label="SeqScan"]
2->3
4[label="Hash"]
2->4
5[label="SeqScan"]
4->5
}'''

In [91]:
node_id = None
lines = g2.split('\n')
for line in lines:
    if 'label="Gather"' in line:
        node_id = line.split(' ')[0]
        break
if node_id is not None:
        for i in range(len(lines)):
            if node_id in lines[i]:
                lines[i] = lines[i].replace(']', ' style=filled fillcolor=red]')
                break
newg = '\n'.join(lines)


In [92]:
newg

'digraph {\n1[label="Gather" style=filled fillcolor=red]\n2[label="HashJoin"]\n1->2\n3[label="SeqScan"]\n2->3\n4[label="Hash"]\n2->4\n5[label="SeqScan"]\n4->5\n}'

In [89]:
node_id

'1[label="Gather"]'

In [90]:
lines

['digraph {',
 '1[label="Gather" style=filled fillcolor=red]',
 '2[label="HashJoin"]',
 '1->2',
 '3[label="SeqScan"]',
 '2->3',
 '4[label="Hash"]',
 '2->4',
 '5[label="SeqScan"]',
 '4->5',
 '}']

In [96]:
def highlight_node(dot_string,element):
    # Find the node with label "Gather"
    node_id = None
    lines = dot_string.split('\n')
    for line in lines:
        if 'label='+str(element) in line:
            node_id = line.split(' ')[0]
            break
    
    # If the node is found, add a red fill color to it
    if node_id is not None:
        for i in range(len(lines)):
            if node_id in lines[i]:
                lines[i] = lines[i].replace(']', ' style=filled fillcolor=red];')
                break
    
    # Return the modified dot string
    return '\n'.join(lines)

In [97]:
highlight_node(g2,'"Gather"')

'digraph {\n1[label="Gather" style=filled fillcolor=red];\n2[label="HashJoin"]\n1->2\n3[label="SeqScan"]\n2->3\n4[label="Hash"]\n2->4\n5[label="SeqScan"]\n4->5\n}'

In [60]:
import pydot
graph1 = pydot.graph_from_dot_data(g1)[0]
graph1_labels = set([node.get_label() for node in graph1.get_nodes()])

In [61]:
graph1_labels

{'"Hash"', '"HashJoin"', '"SeqScan"'}

In [62]:
graph2 = pydot.graph_from_dot_data(g2)[0]
graph2_labels = set([node.get_label() for node in graph2.get_nodes()])

In [63]:
graph2_labels

{'"Gather"', '"Hash"', '"HashJoin"', '"SeqScan"'}

In [64]:
import pydot

def compare_graph_labels(graph1_str, graph2_str):
    # Parse the Graphviz graphs from the input strings
    graph1 = pydot.graph_from_dot_data(graph1_str)[0]
    graph2 = pydot.graph_from_dot_data(graph2_str)[0]

    # Get the labels of the nodes in each graph
    graph1_labels = set([node.get_label() for node in graph1.get_nodes()])
    graph2_labels = set([node.get_label() for node in graph2.get_nodes()])

    # Find the labels present in graph1 but missing in graph2
    missing_in_graph2 = graph1_labels - graph2_labels

    # Find the labels present in graph2 but missing in graph1
    missing_in_graph1 = graph2_labels - graph1_labels

    # Return the missing labels for each graph
    return missing_in_graph1, missing_in_graph2

In [65]:
graph1 ='digraph { 1[label="A"] 2[label="B"] 1->2 }'

graph2 = 'digraph { 1[label="A"] 2[label="C"] 1->2 }'



In [66]:
compare_graph_labels(g1, g2)

({'"Gather"'}, set())

In [47]:
print(f"Labels only in graph 1: {diff1}")
print(f"Labels only in graph 2: {diff2}")

Labels only in graph 1: set()
Labels only in graph 2: set()


In [31]:
compare_graph_labels(graph1,graph2)

AttributeError: 'str' object has no attribute 'graph'

In [26]:
remove_lines_without_arrow(inp)

'Gather\nHashJoin\nHashJoin\nHash\nSeqScan\nHash\nSeqScan\nSeqScan'

In [57]:

import re

query = "select AVG(o_totalprice) from orders where a=b and c=c and orders like chenh"
select_clause = ''
from_clause = ''
where_clause = ''


# Use regular expressions to parse the query string
try:
    select_clause = re.search(r"(?i)^SELECT\s+(.+?)\s+FROM", query).group(1)
except:
    try:
        select_clause = re.search(r"(?i)^SELECT\s+(.+?)$", query).group(1)
    except:
        pass

try:
    from_clause = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)\s+WHERE", query).group(1)
except:
    try:
        from_clause = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)$", query).group(1)
    except:
        pass

try:
    where_clause = re.search(r"(?i)^SELECT.+?\s+FROM.+?\s+WHERE\s+(.+?)$", query).group(1)
except:
    pass


print("SELECT Clause:", select_clause)
print("FROM Clause:", from_clause)
print("WHERE Clause:", where_clause)

SELECT Clause: AVG(o_totalprice)
FROM Clause: orders
WHERE Clause: a=b and c=c and orders like chenh


In [61]:
def query_diff(q1, q2):
    select_clause1 = ''
    from_clause1 = ''
    where_clause1 = ''
    select_clause2 = ''
    from_clause2 = ''
    where_clause2 = ''
    if(q1 == q2):
        return "There are no changes between the queries"
    comp_str = 'due to changes in the '

    try:
        select_clause1 = re.search(r"(?i)^SELECT\s+(.+?)\s+FROM", q1).group(1)
    except:
        try:
            select_clause1 = re.search(r"(?i)^SELECT\s+(.+?)$", q1).group(1)
        except:
            pass

    try:
        from_clause1 = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)\s+WHERE", q1).group(1)
    except:
        try:
            from_clause1 = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)$", q1).group(1)
        except:
            pass

    try:
        where_clause1 = re.search(r"(?i)^SELECT.+?\s+FROM.+?\s+WHERE\s+(.+?)$", q1).group(1)
    except:
        pass

    try:
        select_clause2 = re.search(r"(?i)^SELECT\s+(.+?)\s+FROM", q2).group(1)
    except:
        try:
            select_clause2 = re.search(r"(?i)^SELECT\s+(.+?)$", q2).group(1)
        except:
            pass

    try:
        from_clause2 = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)\s+WHERE", q2).group(1)
    except:
        try:
            from_clause2 = re.search(r"(?i)^SELECT.+?\s+FROM\s+(.+?)$", q2).group(1)
        except:
            pass

    try:
        where_clause2 = re.search(r"(?i)^SELECT.+?\s+FROM.+?\s+WHERE\s+(.+?)$", q2).group(1)
    except:
        pass

    if str(select_clause1) != str(select_clause2):
        comp_str = comp_str + 'SELECT clause, '
    if str(from_clause1) != str(from_clause2):
        comp_str = comp_str + 'FROM clause, '
    if str(where_clause1) != str(where_clause2):
        comp_str = comp_str + 'WHERE clause, '
    comp_str = comp_str + 'of the Evolved Query'

    return comp_str

In [62]:
q1 = 'select AVG(o_totalprice) from orders where o_totalprice > 5'
q2 = 'select AVG(o_totalprice) from orders'
query_diff(q1,q2)

'due to changes in the WHERE clause, of the Evolved Query'

In [60]:
select AVG(o_totalprice) from orders

SyntaxError: invalid syntax (3540676409.py, line 1)

In [ ]:
select * from customer C, orders O where C.c_custkey = O.o_custkey and O.o_totalprice < 100000.0